In [ ]:
import pyodbc
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
conn = pyodbc.connect(
    'Driver={SQL Server};'
    'Server=NTB-DLZN712\SQLEXPRESS;'
    'Database=Projekt_1;'
    'Trusted_Connection=yes;'
)

In [ ]:
sql_query_shipments = """SELECT [Číslo zásilky], [Datum zpracování] FROM zasilky"""
df_shipments = pd.read_sql(sql_query_shipments, conn)
sql_query_working_days = """SELECT * FROM [Seznam pracovních dnů]"""
df_working_days = pd.read_sql(sql_query_working_days, conn)

In [ ]:
df_shipments = df_shipments.rename(columns={'Číslo zásilky': 'Shipments_no', 'Datum zpracování': 'Processing_date'})
df_shipments = df_shipments.groupby('Processing_date')[['Shipments_no']].count().reset_index()
df_shipments['Month/day'] = df_shipments['Processing_date'].dt.strftime('%m/%d')
df_shipments = df_shipments.groupby('Month/day')[['Shipments_no']].mean().round(2).reset_index()


In [ ]:
#promena- streamlit
year = 2022
df_working_days['Year'] = pd.DatetimeIndex(df_working_days['Datum']).year
df_working_days['Month/day'] = df_working_days['Datum'].dt.strftime('%m/%d')
df_working_days['Week_no'] = df_working_days['Datum'].dt.isocalendar().week
df_wd_year = df_working_days[(df_working_days['Year'] == year)]

In [ ]:
df_final_day = pd.merge(df_wd_year, df_shipments, on=['Month/day'])

In [ ]:
df_final_week = df_final_day.groupby('Week_no')[['Shipments_no']].mean().round(2)
df_final_week['Shipments_no'].plot.bar(
    figsize=(30,8),
    color="orange",
    edgecolor="#256354",
    title="Počet zpracovaných zásilek"

);

In [ ]:
#vyselektování kompletních týdnů v roce
week_count = df_final_day['Week_no'].value_counts().sort_index()
df_final_wholeweek = df_final_week.assign(Week_count = week_count)
df_final_wholeweek = df_final_wholeweek[df_final_wholeweek['Week_count'] == 5]
#df_final_week - index je číslo týdne v roce

In [ ]:
#promenne streamlit
whole_week_no = 4 #pocet celých týdnů
mode ='Mód 2' #minimální hodntoy - workoholik mod
days_rest = 5

#kopie prto urceni dnů a týdnů
df_final_wholeweek_copy = df_final_wholeweek.copy()
df_final_day_copy = df_final_day.copy()

In [ ]:
#urceni týdnů
result_week_list = []
for i in range(whole_week_no):
    if mode =='Mód 1':
        result_week = df_final_wholeweek_copy['Shipments_no'].idxmin()
        df_final_wholeweek_copy = df_final_wholeweek_copy.drop(result_week)
        df_final_day_copy.drop(df_final_day_copy[df_final_day_copy['Week_no'] == result_week].index, inplace = True)
        result_week_list.append(result_week)
    else:
        result_week = df_final_wholeweek_copy['Shipments_no'].idxmax()
        df_final_wholeweek_copy = df_final_wholeweek_copy.drop(result_week)
        df_final_day_copy.drop(df_final_day_copy[df_final_day_copy['Week_no'] == result_week].index, inplace = True)
        result_week_list.append(result_week)

In [ ]:
#urceni dnů
result_days_list = []
for i in range(days_rest):
    if mode =='Mód 1':
        result_day_index = df_final_day_copy['Shipments_no'].idxmin()
        result_day = df_final_day_copy.at[result_day_index, 'Datum']
        df_final_day_copy.drop(df_final_day_copy[df_final_day_copy['Datum'] == result_day].index, inplace = True)
        result_day = result_day.date()
        result_days_list.append(result_day)
    else:
        result_day_index = df_final_day_copy['Shipments_no'].idxmax()
        result_day = df_final_day_copy.at[result_day_index, 'Datum']
        df_final_day_copy.drop(df_final_day_copy[df_final_day_copy['Datum'] == result_day].index, inplace = True)
        result_day = result_day.date()
        result_days_list.append(result_day)
        
